In [7]:
from rdflib import Graph
import json

In [8]:
# Create a graph
g = Graph()

# Parse the RDF/XML file
g.parse("food_nutrition.rdf", format="xml")

print(f"Loaded {len(g)} triples.")

http://example.org/food-nutrition#Fish_fatty_>5_g_fat_prepared_in_microwave_oven_no_ingredients_added does not look like a valid URI, trying to serialize this will break.
http://example.org/food-nutrition#Soft_drink_w_sugar_and_sweetener_2-<5_CHO_w_caffeine does not look like a valid URI, trying to serialize this will break.
http://example.org/food-nutrition#Margarine_80%_fat_<24_g_sat_fa_unsalted does not look like a valid URI, trying to serialize this will break.
http://example.org/food-nutrition#Low_fat_margarine_prod_35%_fat_<10_g_sat_fa_unsalted does not look like a valid URI, trying to serialize this will break.
http://example.org/food-nutrition#Pork_>15%_fat_raw_av does not look like a valid URI, trying to serialize this will break.
http://example.org/food-nutrition#Margarine_80%_fat_>24_g_sat_fatty_acids_salted_for_NEVO_recipes does not look like a valid URI, trying to serialize this will break.
http://example.org/food-nutrition#Veal_>5%_fat_raw_av does not look like a valid UR

Loaded 18783 triples.


In [9]:
q = """
PREFIX fn: <http://example.org/food-nutrition#>
SELECT ?foodName ?diet ?benefit ?protein ?carbs ?fat ?vitaminC ?iron ?group ?subgroup
WHERE {
  ?food a fn:Food ;
        fn:foodName ?foodName ;
        fn:belongsToFoodWheelGroup ?group ;
        fn:belongsToFoodWheelSubGroup ?subgroup ;
        fn:belongsToDiet ?diet ;
        fn:hasBenefit ?benefit ;
        fn:hasProtein ?protein ;
        fn:hasCarbohydrates ?carbs ;
        fn:hasFat ?fat ;
        fn:hasVitaminC ?vitaminC ;
        fn:hasIron ?iron .
}
ORDER BY ?group ?subgroup ?foodName
"""

results = g.query(q)

foods = [
    {
        "foodName": str(row.foodName),
        "diet": str(row.diet).split("#")[-1],
        "benefit": str(row.benefit).split("#")[-1],
        "protein": float(row.protein),
        "carbohydrates": float(row.carbs),
        "fat": float(row.fat),
        "vitaminC": float(row.vitaminC),
        "iron": float(row.iron)
    }
    for row in results
]

with open("all_foods.json", "w") as f:
    json.dump(foods, f, indent=2)